## 初期設定

In [1]:
"""環境"""
# .lab

"""更新履歴"""
# 最終更新日：2025/08/04
# 2025/08/04 cachedirの洗浄をオプションにしました。
# 2025/07/22 pickleを保存するために、pickleとdatetimeをインポートするようにしました。
# 2025/07/17 inputsを保存するjsonを作ることにしました
# 2025/07/17 ctypesを使ってtkinterの解像度を上げることにしました
# 2025/05/21 shutilを使って、cachedirを洗浄することにしました
# 2025/05/15 simple_progress_barおよびdata(dict)を使い始めました。
# 2025/02/25 os.getcwd()を採用し、__file__を定義する必要をなくしました。

"""モジュール読み込み"""
# ファイル操作等
import sys
import os
from datetime import datetime
from pprint import pprint
import logging
import pickle
# import struct
from tqdm import tqdm
import h5py
# import threading
import json
import shutil

# tkinter
from tkinter import filedialog, messagebox, Tk
import ctypes
ctypes.windll.shcore.SetProcessDpiAwareness(1) # tkinterの解像度を上げる

# データ分析ツール
import pandas as pd
import numpy as np
import scipy as sp
# import math
# from sklearn.linear_model import LinearRegression

# グラフ等作成用
import matplotlib
import matplotlib.pyplot as plt         # 図の作成用
from PIL import Image as im
# import cv2
from IPython.display import display, HTML, clear_output, update_display, Image

# 自作モジュール
sys.path.append(r"C:\Users\okaza\pythonenv")
from modules.Mytools.Tools import print_fileinfo, h5_tree, dict_tree, simple_progress_bar, clean_cache_except_logfiles, get_total_size
import modules.Mytools.Settings
import modules.fitXRD as fx
from modules.peakfit import peakfit, pseudoVoigt

"""ログ管理ツール作成"""
# chche directoryの設定
cachedir = os.path.abspath(os.getcwd() + "/.cache")
if False:
    clean_cache_except_logfiles(cachedir)
os.makedirs(cachedir, exist_ok=True)

# loggerの作成
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
format = "%(levelname)-9s  %(asctime)s [%(filename)s:%(lineno)d] %(message)s"

# Streamハンドラクラスを作成
sh = logging.StreamHandler()
sh.setLevel(logging.DEBUG)
sh.setFormatter(logging.Formatter(format))
logger.addHandler(sh)

# Fileハンドラクラスをインスタンス化
logfile = cachedir + ""
fh = logging.FileHandler(filename=cachedir + "/notebook.log", encoding="utf-8")
fh.setLevel(logging.DEBUG)
fh.setFormatter(logging.Formatter(format))
logger.addHandler(fh)
logger.debug("[Activate workspace]: " + os.getcwd())
logger.debug("[Set cache]: "+ os.path.abspath(cachedir))
logger.debug("[Activate log]: " + os.path.abspath(cachedir + "/notebook.log"))

# globalなデータを作成
data = dict()
logger.debug("[Create global variable]: data")

# inputをjsonファイルに保存する
data["inputs"] = os.path.join(cachedir, "input.json")
with open(data["inputs"], mode = "w") as f:
    json.dump(dict(), f)
logger.info("[Set json for save inputs]: " + os.path.abspath(data["inputs"]) )

生データの保存

In [181]:
def save_pickle():

    """変更履歴
    * 2025/08/05: defaultextensitionを追加、initialdirの削除
    * 2025/08/01: 作成ののち変更
    """

    # 現在時刻の取得
    dt = datetime.now()
    initfilename = dt.strftime('%Y%m%d%H%M%S%f') + ".pkl"

    # tkinterでファイルダイアログを開く
    window = Tk()
    window.wm_attributes("-topmost", 1)
    window.withdraw()
    filename = filedialog.asksaveasfilename(
        parent = window,
        filetypes = [
            ("pkl", "*.pkl"),
        ],
        initialfile=initfilename,
        defaultextension=".pkl",
    )

    # filenameがセットされたら保存する
    if not (filename == ""):
        with open(filename, mode = "wb") as f:
            pickle.dump(
                obj = data,
                file = f
            )
    logger.info("[Save data]: " + os.path.abspath(filename))

    return
save_pickle()
del save_pickle

生データの読み込み

In [5]:
def load_pickle():

    """変更履歴
    * 2025/08/04: dataの上書きができていませんでした
    * 2025/07/25: 作成
    """

    # tkinterでファイルダイアログを開く
    window = Tk()
    window.wm_attributes("-topmost", 1)
    window.withdraw()
    filename = filedialog.askopenfilename(
        parent = window,
        filetypes = [
            ("pkl", "*.pkl"),
        ],
    )
    global data
    if not (filename == ""):
        with open(filename, mode = "rb") as f:
            data = pickle.load(
                file = f
            )
    logger.info("[Load]: " + os.path.abspath(filename))
    dict_tree(data)

    return
load_pickle()
del load_pickle

## 目的

csvファイルを読み込んでフィッティングを行います。

## 1. ファイルの読み込み

まずファイルリストを作成します。

In [ ]:
def set_filelist():

    # ディレクトリ名を指定
    dir = r"C:\Users\okaza\Box\DataStrorage\2025_06_BL10XU\Okazaki\UODE15\FPD_1"
    
    # ヘッダーとフッターを指定
    header = "UODE15_5_"
    footer = ".csv"

    if True: # Main

        """変更履歴
        * 2025/08/01: 変更履歴作成
        """

        # inputの保存
        json_input = data["inputs"]
        with open(json_input, mode = "r") as f:
            inputs = json.load(f)
        key = sys._getframe().f_code.co_name
        inputs[key] = dict()
        inputs[key]["dir"] = dir
        inputs[key]["header"] = header
        inputs[key]["footer"] = footer
        with open(json_input, mode = "w") as f:
            json.dump(inputs, f, indent = 4)

        # ヘッダーとフッターを含むファイル名を取得
        flist = list()
        for __ in os.listdir(dir):
            if not header in __:
                continue
            if not footer in __:
                continue
            flist.append(__)
        logger.debug("Read filenames")

        # ソート
        flist.sort(key = (lambda x: int(x.replace(header, "").replace(footer, ""))))
        logger.debug("Sort the file name list")

        # 表示
        flist = flist[:442]
        for f in flist:
            print(os.path.abspath(dir + "/" + f))

        # 格納
        key = sys._getframe().f_code.co_name
        data[key] = dict()
        data[key]["dir"] = dir
        data[key]["flist"] = flist
        data[key]["header"] = header
        data[key]["footer"] = footer
        logger.info("[Add variables]: " + key)
        dict_tree(data)

    return
set_filelist()
del set_filelist

データを読み込んで、hdfファイルに保存します。

In [7]:
import threading
import concurrent.futures as confu

In [8]:
def convertCSV2HDF():

    """更新履歴
    * 2025/06/25: データの取得方法を変更
    """

    # hdfファイルを読み込む場合はここを指定する
    # hdffilename = r""

    if not ("hdffilename" in locals()):

        # 変数読み込み
        dir = data["set_filelist"]["dir"]
        flist = data["set_filelist"]["flist"]
        n_frame = len(flist)

        # hdfファイル初期化
        key = sys._getframe().f_code.co_name
        hdffilename = cachedir + "/" + key + ".hdf"
        theta = pd.read_csv(dir + "/" + flist[0], header = None).values.T[0]
        with h5py.File(hdffilename, mode = "w") as f:
            f.create_dataset(
                name = "2theta",
                data = theta,
                shape = theta.shape,
                dtype = theta.dtype
            )
            g = f.create_group(
                name = "intensity"
            )
            for i in tqdm(range(n_frame)):
                g.create_dataset(
                    name = "frame = {}".format(i),
                    shape = theta.shape,
                    dtype = theta.dtype
                )
        
        # lock
        lock = threading.Lock()

        # 演算
        with confu.ThreadPoolExecutor(max_workers=os.cpu_count()) as tpe:
            
            # 演算開始
            futures = [
                tpe.submit(
                    lambda filename: (
                        int(os.path.splitext(filename)[0].replace(data["set_filelist"]["header"], "").replace(data["set_filelist"]["footer"], "")),
                        pd.read_csv(dir + "/" + filename, header = None).values.T[1]),
                    filename
                ) for filename in flist
            ]

            # 終わったプロセスから順に出力
            for i, future in enumerate(confu.as_completed(futures)):
                intensity = future.result()
                with lock:
                    j = intensity[0]
                    with h5py.File(hdffilename, mode = "r+") as f:
                        f["intensity"]["frame = {}".format(j)][:] = intensity[1] # type: ignore
                simple_progress_bar(i+1, n_frame)
    logger.info("[Save hdf]: " + os.path.abspath(hdffilename))

    # 表示
    data[key] = dict()
    data[key]["hdf"] = hdffilename
    logger.info("[Add variables]: " + key)
    with h5py.File(hdffilename, mode = "r") as f:
        h5_tree(f)
    
    return
convertCSV2HDF()
del convertCSV2HDF

## 2. 全データの可視化

1次元データを線画で表示します。

In [188]:
def plot_profile():

    # profileの間隔
    aset = 0.8

    # x軸の範囲
    theta_range = (7,23)

    # figのサイズ
    size_inches = (3,4.5)
    size_inches = (10,4.5)

    # 画像表示モード # "plot" or "imshow"
    mode = "imshow"
    mode = "plot"

    if True: # Main

        """変更履歴
        * 2025/08/01: 変更履歴作成
        """

        # inputの保存
        json_input = data["inputs"]
        with open(json_input, mode = "r") as f:
            inputs = json.load(f)
        key = sys._getframe().f_code.co_name
        inputs[key] = dict()
        inputs["aset"] = aset
        inputs["theta_range"] = theta_range
        inputs["size_inches"] = size_inches
        inputs["mode"] = mode
        with open(json_input, mode = "w") as f:
            json.dump(inputs, f, indent = 4)

        # flag処理
        if True:
            try:
                theta_range # type: ignore
            except:
                theta_range = None
                switch_theta_range = False
            else:
                switch_theta_range = True
        if not mode in ["plot", "imshow"]:
            raise KeyError("mode must be 'plot' or 'imshow'")

        # figureを作成
        fig, ax = plt.subplots()
        fig.set_size_inches(size_inches)
        fig.set_dpi(300)
        fig.subplots_adjust(
            left = 0.08,
            right = 0.92,
            bottom = 0.08,
            top = 0.92
        )

        # xデータを読み込み（共通）
        with h5py.File(data["convertCSV2HDF"]["hdf"], mode = "r") as f:
            theta = np.array(f["2theta"][()]) # type: ignore
        
        # mask処理
        mask = np.ones(theta.shape).astype(np.bool_)
        if switch_theta_range:
            mask[theta<theta_range[0]] = False # type: ignore
            mask[theta>theta_range[1]] = False # type: ignore
        
        # axを構成
        ax.set_xlabel("2theta [degree]", fontsize = 10)
        ax.autoscale(tight = True)
        if switch_theta_range:
            ax.set_xlim(*theta_range) # type: ignore
        else:
            ax.set_xlim(theta[0], theta[-1])
        ax.set_yticks([])

        if mode == "plot":
            # すべてプロットする
            n_frame = len(data["set_filelist"]["flist"])
            with h5py.File(data["convertCSV2HDF"]["hdf"], mode = "r") as f:
                for i in range(n_frame):
                    intensity = np.array(f["intensity"]["frame = {}".format(i)][()]) # type: ignore
                    ax.plot(
                        theta[mask],
                        (intensity + i*aset)[mask],
                        lw = 0.1,
                        c = "0"
                    )
                    simple_progress_bar(i+1, n_frame)

        elif mode == "imshow":
            # すべてをstack
            n_frame = len(data["set_filelist"]["flist"])
            d = []
            with h5py.File(data["convertCSV2HDF"]["hdf"], mode = "r") as f:
                for i in range(n_frame):
                    intensity = np.array(f["intensity"]["frame = {}".format(i)][()]) # type: ignore
                    d.append(intensity[mask])
                    simple_progress_bar(i+1, n_frame)
            d = np.vstack(d)

            ax.imshow(
                d,
                cmap = "gray",
                extent = (theta[mask][0], theta[mask][-1], 0, n_frame),
                origin = "lower",
                aspect = "auto",
                vmin = d.min(),
                vmax = d.max(),
            )

        # 画像の表示
        fig.canvas.draw()
        img = im.frombuffer(
            mode = "RGBA",
            size = fig.canvas.get_width_height(),
            data = fig.canvas.buffer_rgba(), # type: ignore
            decoder_name = "raw"
        )
        key = sys._getframe().f_code.co_name
        imgfilename = cachedir + "/{}.png".format(key)
        img.save(imgfilename)
        logger.debug("[Save fig]: " + os.path.abspath(imgfilename))        
        pdffilename = cachedir + "/{}.pdf".format(key)
        plt.savefig(pdffilename)
        logger.debug("[Save fig]: " + os.path.abspath(pdffilename))
        plt.close()
        display(Image(filename = imgfilename, width = size_inches[0]*100))

    return
plot_profile()
del plot_profile

## 3. フィッティング

In [140]:
from modules.peakfit import peakfit, pseudoVoigt
import matplotlib.animation as anim

フィッティング範囲を絞ります。

In [164]:
def set_fitlim():

    # profileの間隔
    aset = 1

    # x軸の範囲
    theta_range = (16,19)

    # fittingする範囲
    fit_range = (17.5,18.5)

    # グラフのサイズ
    size_inches = (3,5)

    if True: # Main

        """変更履歴
        * 2025/08/01: 変更履歴作成
        """

        # figureを作成
        fig, ax = plt.subplots()
        fig.set_size_inches(size_inches)
        fig.set_dpi(300)
        fig.subplots_adjust(
            left = 0.08,
            right = 0.92,
            bottom = 0.08,
            top = 0.92
        )

        # xデータを読み込み（共通）
        with h5py.File(data["convertCSV2HDF"]["hdf"], mode = "r") as f:
            theta = np.array(f["2theta"][()]) # type: ignore
        
        # mask処理
        mask = np.ones(theta.shape).astype(np.bool_)
        mask[theta < theta_range[0]] = False
        mask[theta > theta_range[1]] = False
        fitmask = np.ones(theta.shape).astype(np.bool_)
        fitmask[theta < fit_range[0]] = False
        fitmask[theta > fit_range[1]] = False

        # すべてプロットする
        n_frame = len(data["set_filelist"]["flist"])
        with h5py.File(data["convertCSV2HDF"]["hdf"], mode = "r") as f:
            for i in range(n_frame):
                intensity = np.array(f["intensity"]["frame = {}".format(i)][()]) # type: ignore
                ax.plot(
                    theta[mask],
                    (intensity + i*aset)[mask],
                    lw = 0.1,
                    c = "0"
                )
                ax.plot(
                    theta[fitmask],
                    (intensity + i*aset)[fitmask],
                    lw = 0.1,
                    c = "tab:orange"
                )
                simple_progress_bar(i+1, n_frame)

        # axを構成
        ax.set_xlabel("2theta [degree]", fontsize = 10)
        ax.autoscale(tight = True)
        ax.set_xlim(*theta_range) # type: ignore
        ylim = ax.get_ylim()

        # 画像の表示
        fig.canvas.draw()
        img = im.frombuffer(
            mode = "RGBA",
            size = fig.canvas.get_width_height(),
            data = fig.canvas.buffer_rgba(), # type: ignore
            decoder_name = "raw"
        )
        key = sys._getframe().f_code.co_name
        imgfilename = cachedir + "/{}.png".format(key)
        img.save(imgfilename)
        logger.debug("[Save fig]: " + os.path.abspath(imgfilename))
        pdffilename = cachedir + "/{}.pdf".format(key)
        plt.savefig(pdffilename)
        logger.debug("[Save fig]: " + os.path.abspath(pdffilename))
        plt.close()
        display(Image(filename = imgfilename, width = size_inches[0]*100))

        # データ格納
        data[key] = dict()
        data[key]["theta_range"] = theta_range
        data[key]["aset"] = aset
        data[key]["size_inches"] = size_inches
        data[key]["fit_range"] = fit_range
        data[key]["ylim"] = ylim
        logger.info("[Add variables]: {}".format(key))
        dict_tree(data)

    return
set_fitlim()
del set_fitlim

In [165]:
def autofit():

    """変更履歴
    * 2025/08/01: 変更履歴作成
    """

    # 角度データを読み込み
    with h5py.File(data["convertCSV2HDF"]["hdf"], mode = "r") as f:
        theta = np.array(f["2theta"][()]) # type: ignore
    
    # 強度データを取得
    n_frame = len(data["set_filelist"]["flist"])
    intensities = list()
    with h5py.File(data["convertCSV2HDF"]["hdf"], mode = "r") as f:
        for i in range(n_frame):
            intensities.append(np.array(f["intensity"]["frame = {}".format(i)][()])) # type: ignore
    intensities = np.vstack(intensities)

    # mask処理
    mask = np.ones(theta.shape).astype(np.bool_)
    mask[theta < data["set_fitlim"]["fit_range"][0]] = False
    mask[theta > data["set_fitlim"]["fit_range"][1]] = False
    theta_fit = theta[mask]
    intensities_fit = intensities.T[mask].T

    # 出力格納用変数を定義
    res = [None]*n_frame

    # フィット用関数を定義
    pf = peakfit()
    def process(i):
        _res = pf.fit_Vigot_func(theta = theta_fit, intensity = intensities_fit[i])
        res_dict = dict()
        for j, k in enumerate(["popt", "pcov"]):
            res_dict[k] = dict()
            for l, m in enumerate(pf.variables(nop = 1)):
                res_dict[k][m] = _res[j][l]
        res_dict["r2"] = _res[2]
        return i, res_dict

    # マルチスレッドを使いながら演算
    with confu.ThreadPoolExecutor() as tpe:
        futures = [tpe.submit(process, i_frame) for i_frame in np.arange(n_frame)]
        for i, future in enumerate(confu.as_completed(futures)):
            i_frame, val = future.result()
            res[i_frame] = val # type: ignore
            simple_progress_bar(i+1, n_frame)
    
    # データ格納
    key = sys._getframe().f_code.co_name
    data[key] = dict()
    data[key]["res"] = res
    logger.debug("[Add variable]: res")
    logger.info("size of data: {} MB".format(sys.getsizeof(data)%1024%1024))

    # データ保存
    jsonfile = os.path.join(cachedir, key+".json")
    with open(jsonfile, mode = "w") as f:
        json.dump(res, f, indent = 4)
    logger.info("[Save json]: " + os.path.abspath(jsonfile))
    dict_tree(data)

    return
autofit()
del autofit

フィッティング結果を出力します。

In [166]:
def plot_res():


    """変更履歴
    * 2025/08/01: 変更履歴作成
    """

    # 変数を読み込み
    theta_range = data["set_fitlim"]["theta_range"]
    fit_range = data["set_fitlim"]["fit_range"]
    aset = data["set_fitlim"]["aset"]
    size_inches = data["set_fitlim"]["size_inches"]
    ylim = data["set_fitlim"]["ylim"]

    # figureを作成
    fig, ax = plt.subplots()
    fig.set_size_inches(size_inches)
    fig.set_dpi(300)
    fig.subplots_adjust(
        left = 0.08,
        right = 0.92,
        bottom = 0.08,
        top = 0.92
    )

    # xデータを読み込み（共通）
    with h5py.File(data["convertCSV2HDF"]["hdf"], mode = "r") as f:
        theta = np.array(f["2theta"][()]) # type: ignore
    
    # mask処理
    mask = np.ones(theta.shape).astype(np.bool_)
    mask[theta < theta_range[0]] = False
    mask[theta > theta_range[1]] = False
    fitmask = np.ones(theta.shape).astype(np.bool_)
    fitmask[theta < fit_range[0]] = False
    fitmask[theta > fit_range[1]] = False

    # すべてプロットする
    n_frame = len(data["set_filelist"]["flist"])
    pf = peakfit()
    with h5py.File(data["convertCSV2HDF"]["hdf"], mode = "r") as f:
        for i in range(n_frame):
            intensity = np.array(f["intensity"]["frame = {}".format(i)][()]) # type: ignore
            ax.plot(
                theta[mask],
                (intensity + i*aset)[mask],
                lw = 0.1,
                c = "0"
            )
            x = np.linspace(theta[fitmask][0], theta[fitmask][-1], 200)
            y: np.ndarray = pseudoVoigt(
                x,
                *[data["autofit"]["res"][i]["popt"][j] for j in pf.variables()]
            ) # type: ignore
            ax.plot(
                x,
                y + i*aset,
                lw = 0.1,
                c = "tab:orange"
            )
            ax.plot(
                [data["autofit"]["res"][i]["popt"]["mu"]],
                [
                    data["autofit"]["res"][i]["popt"]["amp"]
                    + data["autofit"]["res"][i]["popt"]["b0"]
                    + data["autofit"]["res"][i]["popt"]["b1"] * data["autofit"]["res"][i]["popt"]["mu"]
                    + i*aset
                ],
                lw = 0,
                marker = "o",
                ms = 1,
                mec = "tab:orange",
                c = "1",
                mew = 0.1
            )
            simple_progress_bar(i+1, n_frame)

    # axを構成
    ax.set_xlabel("2theta [degree]", fontsize = 10)
    ax.autoscale(tight = True)
    ax.set_xlim(*theta_range) # type: ignore
    ax.set_ylim(ylim)

    # 画像の表示
    fig.canvas.draw()
    img = im.frombuffer(
        mode = "RGBA",
        size = fig.canvas.get_width_height(),
        data = fig.canvas.buffer_rgba(), # type: ignore
        decoder_name = "raw"
    )
    key = sys._getframe().f_code.co_name
    imgfilename = cachedir + "/{}.png".format(key)
    img.save(imgfilename)
    logger.debug("[Save fig]: " + os.path.abspath(imgfilename))
    pdffilename = cachedir + "/{}.pdf".format(key)
    plt.savefig(pdffilename)
    logger.debug("[Save fig]: " + os.path.abspath(pdffilename))
    plt.close()
    display(Image(filename = imgfilename, width = size_inches[0]*100))

    return
plot_res()
del plot_res

フィッティング結果の可視化をします。

In [167]:
def show_fitting_result():

    """変更履歴
    * 2025/08/04: 作成
    """

    # jsonファイルを読み込む
    key = "autofit"
    jsonfile = cachedir + "/{}.json".format(key)
    with open(jsonfile, mode = "r") as f:
        j = json.load(f)
    logger.info("[Load json]: " + os.path.abspath(jsonfile))
        

    # データフレームを作成
    x = np.arange(len(j))
    mu = [j[i]["popt"]["mu"] for i in range(len(j))]
    err_mu = np.sqrt(np.array([j[i]["pcov"]["mu"] for i in range(len(j))]))
    fwhm_g = np.array([j[i]["popt"]["fwhm_g"] for i in range(len(j))])
    fwhm_l = np.array([j[i]["popt"]["fwhm_l"] for i in range(len(j))])
    eta = np.array([j[i]["popt"]["eta"] for i in range(len(j))])
    r2 = np.array([j[i]["r2"] for i in range(len(j))])

    
    # figureを作成
    fig = plt.figure()
    size_inches = (8,12)
    fig.set_size_inches(size_inches)
    fig.set_dpi(300)

    # mu
    ax_mu = fig.add_axes(
        rect = (
            0.1,
            0.7375,
            0.8,
            0.2
        )
    )
    ax_mu.errorbar(
        x,
        mu,
        yerr = err_mu,
        lw = 0,
        fmt = "o",
        ms = 6,
        elinewidth = 1,
        ecolor = "0",
        mfc = "1",
        mec = "0"
    )
    ax_mu.set_ylabel("Peak position [deg]", fontsize = 10)
    ylim_mu = ax_mu.get_ylim()
    range_mu = ylim_mu[1]-ylim_mu[0]
    ax_mu.set_xticklabels([])

    # fwhm
    ax_fwhm = fig.add_axes(
        rect = (
            0.1,
            0.5125,
            0.8,
            0.2
        )
    )
    ax_fwhm.errorbar(
        x,
        fwhm_g,
        # yerr = err_fwhm_g,
        lw = 0,
        fmt = "^",
        ms = 6,
        elinewidth = 1,
        ecolor = "tab:blue",
        mfc = "1",
        mec = "tab:blue",
        label = "gaussian"
    )
    ax_fwhm.errorbar(
        x,
        fwhm_l,
        # yerr = err_fwhm_l,
        lw = 0,
        fmt = "v",
        ms = 6,
        elinewidth = 1,
        ecolor = "tab:orange",
        mfc = "1",
        mec = "tab:orange",
        label = "lorentzian"
    )
    ax_fwhm.set_ylim((0, range_mu*20))
    ax_fwhm.legend()
    ax_fwhm.set_ylabel("Full width at half maximum [deg]", fontsize = 10)
    ax_fwhm.set_xticklabels([])

    # eta
    ax_eta = fig.add_axes(
        rect = (
            0.1,
            0.2875,
            0.8,
            0.2
        )
    )
    ax_eta.errorbar(
        x,
        eta,
        # yerr = err,
        lw = 0,
        fmt = "o",
        ms = 6,
        elinewidth = 1,
        ecolor = "0",
        mfc = "1",
        mec = "0"
    )
    ax_eta.set_ylabel("Gaussian rate", fontsize = 10)
    ax_eta.set_ylim(-0,1)
    ax_eta.set_xticklabels([])

    # fitting
    ax_r2 = fig.add_axes(
        rect = (
            0.1,
            0.0625,
            0.8,
            0.2
        )
    )
    ax_r2.errorbar(
        x,
        r2,
        # yerr = err,
        lw = 0,
        fmt = "o",
        ms = 6,
        elinewidth = 1,
        ecolor = "0",
        mfc = "1",
        mec = "0"
    )
    ax_r2.set_ylabel("Determination coefficient", fontsize = 10)
    ax_r2.set_ylim(0.8,1)
    ax_r2.set_xlabel("Frame", fontsize = 10)

    # 画像の表示
    fig.canvas.draw()
    img = im.frombuffer(
        mode = "RGBA",
        size = fig.canvas.get_width_height(),
        data = fig.canvas.buffer_rgba(), # type: ignore
        decoder_name = "raw"
    )
    key = sys._getframe().f_code.co_name
    imgfilename = cachedir + "/{}.png".format(key)
    img.save(imgfilename)
    logger.debug("[Save fig]: " + os.path.abspath(imgfilename))
    # pdffilename = cachedir + "/{}/{}.pdf".format(peakname, key)
    # plt.savefig(pdffilename)
    # logger.debug("[Save fig]: " + os.path.abspath(pdffilename))
    plt.close()
    display(Image(filename = imgfilename, width = size_inches[0]*100))

    return
show_fitting_result()
del show_fitting_result

csvで保存します

In [176]:
def save_csv():

    """変更履歴
    * 2025/08/05: セル出力を増やしました
    * 2025/08/01: 変更履歴作成
    """

    # jsonファイルを読み込む
    key = "autofit"
    jsonfile = cachedir + "/{}.json".format(key)
    with open(jsonfile, mode = "r") as f:
        j = json.load(f)
    logger.info("[Load json]: " + os.path.abspath(jsonfile))
        
    # データフレームを作成
    mu = [j[i]["popt"]["mu"] for i in range(len(j))]
    err_mu = np.sqrt(np.array([j[i]["pcov"]["mu"] for i in range(len(j))]))
    fwhm_g = [j[i]["popt"]["fwhm_g"] for i in range(len(j))]
    fwhm_l = [j[i]["popt"]["fwhm_l"] for i in range(len(j))]
    eta = [j[i]["popt"]["eta"] for i in range(len(j))]
    r2 = [j[i]["r2"] for i in range(len(j))]
    df = pd.DataFrame(dict(
        twotheta = mu,
        err_twotheta = err_mu,
        fwhm_g = fwhm_g,
        fwhm_l = fwhm_l,
        eta = eta,
        r2 = r2,
    ))
    logger.info("pd.DataFrame made")
    
    # データ保存
    key = sys._getframe().f_code.co_name
    csvfile = os.path.join(cachedir, key + ".csv")
    df.to_csv(csvfile)
    logger.info("[Save csv]: " + os.path.abspath(csvfile))

    # 出力
    display(df.head())
    df.info()

    return
save_csv()
del save_csv

## 4. エクスポート

In [177]:
import subprocess

In [178]:
def export_html():

    """変更履歴
    * 2025/08/01: 変更履歴作成
    """

    # ipynbファイルを検知
    dirname = os.path.dirname(cachedir)
    for i in os.listdir(dirname):
        if os.path.splitext(i)[1] == ".ipynb":
            notebook_name = os.path.join(dirname, i)
            break
    logger.info("[Find notebook]: " + os.path.abspath(notebook_name))

    # htmlに変換
    subprocess.run(["jupyter", "nbconvert", "--to", "html", notebook_name])
    logger.info("[Subprocess]: Export as html file")

    # データ格納
    key = sys._getframe().f_code.co_name
    data[key] = dict()
    data[key]["ipynb"] = notebook_name
    logger.info("size of data: {} MB".format(sys.getsizeof(data)%1024%1024))
    dict_tree(data)

    return
export_html()
del export_html


データを保存します

In [180]:
def save_data():

    """変更履歴
    * 2025/08/01: 変更履歴作成
    """
    
    # tkinterでファイルダイアログを開く
    window = Tk()
    window.wm_attributes("-topmost", 1)
    window.withdraw()
    initialfilename = os.path.basename(os.path.splitext(data["export_html"]["ipynb"])[0]) + ".dir"
    filename = filedialog.asksaveasfilename(
        parent = window,
        filetypes = [
            ("DIRECTORY", "*.dir"),
            ("csv", "*.csv"),
            ("json", "*.json"),
            ("html", "*.html"),
        ],
        initialfile=initialfilename,
        defaultextension=".dir"
    )
    print(filename)
    

    # # 分岐
    if os.path.splitext(filename)[1] == ".csv":
        original = os.path.join(cachedir, "save_csv.csv")
        logger.debug("Original file being copied...: " + os.path.abspath(original))
        shutil.copy2(
            src = original,
            dst = filename
        )
        logger.info("File copied: " + os.path.abspath(filename))

    elif os.path.splitext(filename)[1] == ".json":
        original = os.path.join(cachedir, "autofit.json")
        logger.debug("Original file being copied...: " + os.path.abspath(original))
        shutil.copy2(
            src = original,
            dst = filename
        )
        logger.info("File copied: " + os.path.abspath(filename))

    elif os.path.splitext(filename)[1] == ".html":
        original = os.path.splitext(data["export_html"]["ipynb"])[0] + ".html"
        logger.debug("Original file being copied...: " + os.path.abspath(original))
        shutil.copy2(
            src = original,
            dst = filename
        )
        logger.info("File copied: " + os.path.abspath(filename))

    elif os.path.splitext(filename)[1] == ".dir":
        dirname = os.path.splitext(filename)[0]
        logger.debug("Original directory being copied...: " + os.path.abspath(os.getcwd()))
        if os.path.exists(dirname):
            shutil.rmtree(dirname)
        shutil.copytree(
            src = os.getcwd(),
            dst = dirname,
            dirs_exist_ok = True
        )
        for f in os.listdir(dirname):
            if os.path.splitext(f)[1] == ".ipynb":
                os.remove(os.path.join(dirname, f))
        logger.info("Directory copied: " + os.path.abspath(dirname))

    return
save_data()
del save_data